In [168]:
# General imports
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

In [170]:
from qiskit import *
from qiskit.visualization import *
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session
from qiskit.primitives import Sampler
# Loading IBM Quantum account
service = QiskitRuntimeService(channel="ibm_quantum", token="8558f42abd5ce4c37e51ce37375293303fab09a45f771700ee214028841a7a19c5a8eedca85b84770d7d51177d9869776d88e8adb34bbf0a91b6c607a2e5d1a7")
backend = service.least_busy(operational=True, simulator=False)

In [693]:
############################################################# VARIABLES #############################################################################
B = 200 # Budget
R = 0.5 # Risk Ratio
delta = 1 # Penalty Term

In [678]:
######################################################### IMPORT DATA HERE ##########################################################################
# panda #
import pandas as pd
df = pd.read_csv('sample.csv')

holder = []
for i in range(1,5):
    holder.append(df.iloc[:, i].tolist())

holder2 = []
for i in holder:
    holder2.append(i[0:int(len(holder[0])/24)])

map_everything_for_me_please_here_are_all_the_prices(holder2)
rev_the_hamiltonian()

In [711]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.circuit import Parameter, QuantumCircuit

with Session(backend=backend) as session:
    E = []
    size = sum(d)
    x0 = 2 * np.pi * np.random.random(size*8)
    start_time = time.time()
    cobyla = minimize(cost_funct, x0, method='COBYLA', options={'disp': True}) #'maxiter':400
    opt_params = cobyla.x
    opt_energy = cobyla.fun
    
    end_time = time.time()
    print("time for operation", end_time-start_time)
    opt_x = fetch_investment_vector(opt_params)
    print("optimal vector:", opt_x)
    print("money invested:", np.dot(P, opt_x))

iteration 1 Energy: [46.4772716]
part 1: [0.00455452]
part 2: [-1.34046081]
part 3: [64.15638063]
iteration 2 Energy: [53.45693317]
part 1: [0.00445533]
part 2: [-1.28658301]
part 3: [56.56329616]
iteration 3 Energy: [46.63737847]
part 1: [0.00455219]
part 2: [-1.33543206]
part 3: [63.97281979]
iteration 4 Energy: [45.31738267]
part 1: [0.00457201]
part 2: [-1.30741745]
part 3: [65.5448714]
iteration 5 Energy: [45.05899425]
part 1: [0.00457595]
part 2: [-1.29827257]
part 3: [65.85971091]
iteration 6 Energy: [45.55585654]
part 1: [0.00457537]
part 2: [-0.94228744]
part 3: [65.47815071]
iteration 7 Energy: [44.8108263]
part 1: [0.00457329]
part 2: [-1.91180238]
part 3: [65.78940779]
iteration 8 Energy: [44.97120374]
part 1: [0.00455022]
part 2: [-1.30693879]
part 3: [65.95994822]
iteration 9 Energy: [45.95174508]
part 1: [0.00469076]
part 2: [-1.07030345]
part 3: [64.9322346]
iteration 10 Energy: [44.5135923]
part 1: [0.00477742]
part 2: [-1.81975482]
part 3: [66.2026464]
iteration 11 En

In [713]:
def cost_funct(params):
    q = QuantumRegister(size)
    qc = QuantumCircuit(q)
    
    ######################### ANSATZ LIVES HERE #######################################
    ansatz = EfficientSU2(size)    
    qc.compose(ansatz, inplace=True)
    ##################################################################################
    
    observables = [
    [SparsePauliOp(hi, h_)], [SparsePauliOp(ji, J__)], [SparsePauliOp(bi, pi_)]
    ]
    
    estimator = StatevectorEstimator()
    pub = (qc, observables, params)
    job = estimator.run([pub])
    result = job.result()

    qc.draw('mpl')
    energy_cost = result[0].data.evs[0] + R * result[0].data.evs[1] + delta * (result[0].data.evs[2] - (2-sum(pi_)*(1/B)))**2
    E.append(energy_cost)
    
    return energy_cost

In [676]:
######################################################### GENERATING HAMILTONIAN ####################################################################

#####################################################################################################################################################
def rev_the_hamiltonian():
    global h, h_, hi, J, J_, J__, ji, pi, pi_, bi
    I = []
    for i in range(sum(d)):
        I.append("I")
    
    ############### hi Zi ################
    h = (np.array([u])*np.array([P])/B)
    h_ = []
    for i in range(len(d)):
        for j in range(d[i]):
            h_.append(h[0][i]*(2**j))
        
    hi = []
    for i in range(sum(d)):
        I[i] = "Z"
        hi.append(stitch(I))
        I[i] = "I"
    ######################################
    
    ############# ji Zi Zj ###############
    J = ((convolu*(np.array([P])*(1/B)).reshape(-1, 1)).T)*(np.array([P])/B).reshape(-1, 1)
    J_ = np.dot(conver.T,(np.dot(J, conver)))
    
    J__ = []
    ji = []
    for i in range(sum(d)):
        for j in range(sum(d)):
            I[i] = "Z"
            I[j] = "Z"
            if j != i:
                ji.append(stitch(I))
                J__.append(J_[i,j])
            I[i] = "I"
            I[j] = "I"
    ######################################
    
    ########### Budget Penalty ###########
    pi = (np.array([P])*1/B)
    pi_ = []
    for i in range(len(d)):
        for j in range(d[i]):
            pi_.append(pi[0][i]*(2**j))

    bi = copy.copy(hi)
    ######################################

In [581]:
def map_everything_for_me_please_here_are_all_the_prices(holder):
    global p, P, r, u, convolu, d, conver
    
    # PRICE
    p = []
    for i in holder:
        p.append(i)
        
    # CURRENT PRICE
    P = []
    for i in holder:
        P.append(i[-1])
    
    # REVENUE
    r = []
    for i in holder:
        r.append(reven(i))

    # AVERAGE REVENUE
    u = []
    for i in r:
        u.append(avg(i))

    # CONVOLUTION MATRIX
    convolu = np.zeros((len(holder), len(holder)))

    for i in range(len(holder)):
        for j in range(len(holder)):
            convolu[i,j] = inner(add(r[i],-u[i]), add(r[j],-u[j]))

    # CONVERT TO BITS
    d = []
    for i in range(len(holder)):
        d.append(round(np.log2(int(B/P[i])+1)))

    # REVERSION MATRIX
    conver = np.zeros((len(d), sum(d)))
    
    k = 0
    for i in range(len(d)):
        for j in range(d[i]):
            conver[i,k] = 2**j
            k += 1

In [579]:
import copy

def norm(b):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] /= sum(v)
    return v, sum(v)

def avg(x):
    avg = sum(x)/len(x)
    return avg

def reven(v):
    b = []
    for i in range(1,len(v)):
        b.append(v[i]-v[i-1])
    return b

def add(b, r):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] += r
    return v

def multip(b, r):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] *= r
    return v

def inner(a, b):
    total = 0
    for i in range(len(a)):
        total += a[i] * b[i]
    return total

def stitch(v):
    s = ""
    for i in v:
        s += i
    return s

In [577]:
def dict_to_vec(dict):
    keys_list = list(dict.keys())

    vector = []
    for i in range(2**sum(d)):
        vector.append(dict.get(i, 0))
        
    return vector

In [630]:
def fetch_investment_vector(params):
    with Session(backend=backend) as session:
        ansatz = EfficientSU2(size)
        ansatz.measure_all()
        
        pub = (ansatz, params)
        sampler = StatevectorSampler()
        
        shots = (sum(d))*1000
        job = sampler.run([pub], shots=shots)
        result = job.result()[0]
        
        dict = result.data.meas.get_int_counts()
        max_key = max(dict, key=dict.get)
        
        binary = bin(max_key)[2:]
        binary_ = "0"*(sum(d)-len(binary))+binary
    
        tempvec = np.zeros((1,sum(d)))
        for i in range(sum(d)):
            tempvec[0,i] += int(binary_[i])
        
        xvec = np.dot(conver, tempvec[0])
        return xvec